In [1]:
#| code-fold: true
################################################################################

# autoreload all modules every time before executing the Python code
%reload_ext autoreload
%autoreload 2

################################################################################

from IPython.core.interactiveshell import InteractiveShell

# `ast_node_interactivity` is a setting that determines how the return value of the last line in a cell is displayed
# with `last_expr_or_assign`, the return value of the last expression is displayed unless it is assigned to a variable
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

The `holoviz` ecosystem is a powerful set of libraries that can help build interactive dashboards and UIs. The `param` library is a core piece of the `holoviz` ecosystem. Understanding how this library underneath it all works can help you build more interactive, flexible, and reusable code. In this post, we will build a tiny param library to understand how things works.

## Background

You may have already heard that in Python, pretty much _everything_ is an object. What this means in practice is that in Python, there exists an allocated piece of memory which represents some data that has an address and a label that points to that address.

In [33]:
x = 123456789

123456789

In [34]:
type(x)

int

In this statement above, `x` is a "label" that points to the "memory address of an object" that contains the value `123456789` and the object that `x` points to is of type `int`. 

```mermaid
graph TD
    subgraph CPython_Object["CPython Object"]
        typePointer["Type Pointer"]
        refCount["Reference Count"]
        value["Value: 123456789"]
    end

    x["x (Label)"] --> CPython_Object
    CPython_Object --> typePointer
    CPython_Object --> refCount
    CPython_Object --> value

```

In this post you'll see language like "`x` is a "label" to the value `123456789`", but most commonly, people refer to `x` as a **variable**.

The **address** of a "label" or "variable" can be found using the `id()` function.

In [35]:
hex(id(x))

'0x111c50dd0'

Notice that if we create overwrite the `x` label with an assignment to a _new_ object with the same value, the `id()` function will return a different address.

In [36]:
x = 123456789

123456789

In [37]:
hex(id(x))

'0x111c50f90'

```mermaid
graph TD
    subgraph CPython_Object1["CPython Object"]
        typePointer1["Type Pointer"]
        refCount1["Reference Count"]
        value1["Value: 123456789"]
    end

    subgraph CPython_Object2["CPython Object"]
        typePointer2["Type Pointer"]
        refCount2["Reference Count"]
        value2["Value: 123456789"]
    end

    x1["x (Label)"] --> CPython_Object1
    CPython_Object1 --> typePointer1
    CPython_Object1 --> refCount1
    CPython_Object1 --> value1

    x2["x (Label)"] --> CPython_Object2
    CPython_Object2 --> typePointer2
    CPython_Object2 --> refCount2
    CPython_Object2 --> value2

```

But if we create a _new_ `y` binding to the _same_ object, the `id()` function will return the same address.

In [38]:
y = x

123456789

In [39]:
hex(id(y))

'0x111c50f90'

In [40]:
hex(id(x)) == hex(id(y))

True

```mermaid
graph TD
    subgraph CPython_Object["CPython Object"]
        typePointer["Type Pointer"]
        refCount["Reference Count"]
        value["Value: 123456789"]
    end

    x["x (Label)"] --> CPython_Object
    CPython_Object --> typePointer
    CPython_Object --> refCount
    CPython_Object --> value

    y["y (Label)"] --> CPython_Object

```

::: callout-note

Interestingly, Python caches small integers and strings, so the `id()` function will return the same address for small integers and strings.


In [42]:
x = 42
y = 42
hex(id(x)) == hex(id(x))

True

In [43]:
x = 123456789
y = 123456789
hex(id(x)) == hex(id(y))

False


:::

This example shows integers, because even though they are immutable and even though they are built-in types and one of most basic units of data, they are still "objects" in Python. Pretty much everything in python is an "object".

Most commonly, when people say "object" in Python, they are referring to "instances of a class". 

In [44]:
class Foo:
    ...

f = Foo()

Python prints the address of the object when you print an instance of a class.

In [45]:
hex(id(f))

'0x111c3ba40'

In Python, classes, modules, functions, and methods are all objects. They all are "labels" or "variables" that point to an address in memory.

In [48]:
def my_func():
    print("hello world")

In [49]:
hex(id(my_func))

'0x111c33560'

And in Python, we can assign a new "label" or a new "variable" to the same function object.

In [50]:
x = my_func

<function __main__.my_func()>

In [51]:
x()

hello world


The other key piece of background information is that in Python, when calling a function, the arguments are "passed by reference" and assigned to the variables in the arguments of function. This means that when you pass an argument to a function, you are passing the address of the object that the argument points to. 

In [52]:
def print_id_of_arg(arg):
    print(hex(id(arg)))

In [53]:
print_id_of_arg(my_func)

0x111c33560


This is equivalent to executing the following code:



In [56]:
arg = my_func
print(hex(id(arg)))

0x111c33560


And it doesn't matter what the name of the label is, only the address of the object that the label points to, and the object and type of the object at that address is important.

In [57]:
print_id_of_arg(x)

0x111c33560


With that background, let's build a tiny param library to understand how it works.

## Callbacks